# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
from scipy.stats import mode
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

In [3]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [5]:
class Leaf:
    def __init__(self, y):
        self.y = y
        self.classes, self.samples = np.unique(y, return_counts=True)
        self.predicted = self.classes[np.argmax(self.samples)]


class Node:
    def __init__(self, split_dim, left, right):
        self.split_dim = split_dim
        self.left = left
        self.right = right




class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):

        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.criterion = entropy if criterion == 'entropy' else gini
        bagging_indexes = np.random.choice(X.shape[0], X.shape[0], replace=True)
        self.bagging_X, self.bagging_y = X[bagging_indexes], y[bagging_indexes]
        self.out_of_bag_X = np.delete(X, bagging_indexes, axis=0)
        self.out_of_bag_y = np.delete(y, bagging_indexes)



    def build_node(self, X, y, depth=0):

        if self.max_depth is not None and depth >= self.max_depth:
            return Leaf(y)

        n_samples, n_features = X.shape
        if self.max_features != 'auto':
            k = int(np.sqrt(n_features))
            features = np.random.choice(n_features, k, replace=False)
        else:
            features = np.arange(n_features)
        # As we have only binary features {0, 1} we split everything on feature == 1 and not
        split_dim, max_gain = None, 0.0
        for feature in features:
            mask = X[:, feature] == 0

            could_be_leaf = mask.sum() > self.min_samples_leaf and n_samples - mask.sum() > self.min_samples_leaf
            if could_be_leaf:
                info_gain = gain(y[mask], y[~mask], criterion=self.criterion)
                if info_gain > max_gain:
                    split_dim, max_gain = feature, info_gain
        if split_dim is None:
            return Leaf(y)

        mask = X[:, split_dim] == 0
        left = self.build_node(X[mask], y[mask], depth + 1)
        right = self.build_node(X[~mask], y[~mask], depth + 1)
        return Node(split_dim, left, right)

    def predict(self, X, node=None):
        result = np.empty(X.shape[0], dtype=np.object)

        if node is None:
            node = self.build_node(self.bagging_X, self.bagging_y)
        if isinstance(node, Leaf):
            result[:] = node.predicted
            return result
        else:
            mask = X[:, node.split_dim] == 0
            result[mask] = self.predict(X[mask], node.left)
            result[~mask] = self.predict(X[~mask], node.right)
            return result

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [6]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None,
                 min_samples_leaf=1, max_features="auto", n_estimators=10):

        self.n_estimators = n_estimators
        self.trees = None
        self.params = {'criterion': criterion, 'max_depth': max_depth,
                       'min_samples_leaf': min_samples_leaf, 'max_features': max_features}



    def fit(self, X, y):
        self.trees = [DecisionTree(X, y, **self.params) for _ in range(self.n_estimators)]

    def predict(self, X):
        self.n_features = X.shape[1]
        predicted_trees = np.array([tree.predict(X) for tree in self.trees]).T
        hard_votes = np.array([mode(votes)[0][0] for votes in predicted_trees])
        return hard_votes

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [7]:
def feature_importance(rfc):
    # Внешний цикл деревья, внутренний фичи
    n_trees, n_features = rfc.n_estimators, rfc.n_features

    importance_matrix = np.empty((n_features, n_trees), dtype=np.float64)
    estimators = rfc.trees

    for tree_n in range(n_trees):
        tree = estimators[tree_n]
        X_out, y_out = tree.out_of_bag_X, tree.out_of_bag_y
        err_oob = np.mean(y_out == tree.predict(X_out))
        for feature in range(n_features):
            shuffled_out = np.copy(X_out)
            shuffled_out[:, feature] = np.random.permutation(X_out[:, feature])
            err_oob_sh = np.mean(y_out == tree.predict(shuffled_out))
            importance_matrix[feature, tree_n] = err_oob - err_oob_sh
    return np.mean(importance_matrix, axis=1)


def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [ ]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", *feature_importance(rfc))

### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [9]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [13]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.)

#### Возраст

In [14]:
rfc = RandomForestClassifier(n_estimators=2)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
names = feature_importance(rfc)
names
# print("Most important features:")
# for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
#     print(str(i+1) + ".", name)

Accuracy: 0.5610410094637224


KeyboardInterrupt: 

#### Пол

In [ ]:
rfc = RandomForestClassifier(n_estimators=10)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)

### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [ ]:
X, y = synthetic_dataset(1000)
print("Accuracy:", np.mean(None == y))
print("Importance:", None)

### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [ ]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [41]:
print("Accuracy:", np.mean(None == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 10)):
    print(str(i+1) + ".", name)

Accuracy: 0.0
Most important features:


KeyboardInterrupt: 

#### Пол

In [ ]:
print("Accuracy:", np.mean(None == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(None, features, 10)):
    print(str(i+1) + ".", name)